In [5]:
#Import all the necessary packages that I will use for this project

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [6]:
from bs4 import BeautifulSoup

In [7]:
# GET request to scrape data from wikipedia
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [8]:
# create the empty lists to store info from wikipedia
postalcodes = []
boroughs = []
neighborhoods = []

In [9]:
# Create beautifulsoup object to parse HTML data
soup = BeautifulSoup(data, 'html.parser')

In [10]:
# ask Soup to find the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [11]:
# add the data in the table to our empty lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalcodes.append(cells[0].text.rstrip('\n'))
        boroughs.append(cells[1].text.rstrip('\n'))
        neighborhoods.append(cells[2].text.rstrip('\n'))

In [12]:
# create a new DataFrame from the three lists
TorontoDF = pd.DataFrame({"PostalCode": postalcodes,
                           "Borough": boroughs,
                           "Neighborhood": neighborhoods})

TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
# In order to satisfy the above requirement, I will use the drop command from pandas to drop any value with a borough that is not assigned

TorontoDF = TorontoDF[TorontoDF.Borough != "Not assigned"].reset_index(drop=True)
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
# For any neighborhood that has the value 'Not assigned' replace that value with the Borough instead
for index, row in TorontoDF.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
#Get coordinate data of postal codes
latlong = pd.read_csv("http://cocl.us/Geospatial_data")
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# Change name of column to match Toronto_df
latlong.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
# merge two table on the column "PostalCode"
TorontoDF = TorontoDF.merge(latlong, on="PostalCode", how="left")
TorontoDF.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [18]:
# Load my foursquare credentials to use the API

CLIENT_ID = '31OIH0QK2KJZDWMLAEUPETODOP05CK1Q22050KU3OXPU0QXT'
CLIENT_SECRET = 'CXTSYGJMIBJIKREJZ40Y00SFCISR3R0DWG20CHQGEECBLQFM'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 31OIH0QK2KJZDWMLAEUPETODOP05CK1Q22050KU3OXPU0QXT
CLIENT_SECRET:CXTSYGJMIBJIKREJZ40Y00SFCISR3R0DWG20CHQGEECBLQFM


In [19]:
#create a list of venues within 500 meters of each neighborhood, max of 100 venues

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venues in venues for item in venues])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
# Get venues into a dataframe

venuesDF = getNearbyVenues(names=TorontoDF['Neighborhood'],
                                   latitudes=TorontoDF['Latitude'],
                                   longitudes=TorontoDF['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [21]:
# take a look
venuesDF.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
8,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


In [22]:
#check total number of venues in each neighborhood
Total = venuesDF["Neighborhood"].value_counts().to_frame()

In [23]:
Total.head()

,Neighborhood
"First Canadian Place, Underground city",100
"Commerce Court, Victoria Hotel",100
"Richmond, Adelaide, King",100
"Garden District, Ryerson",100
"Harbourfront East, Union Station, Toronto Islands",100


In [24]:
#Clean up the Total dataframe with accurate column titles
Total = Total.reset_index()

In [26]:
Total = Total.rename(columns={"Neighborhood":"Total"})
Total.head()

,index,Total
0,"First Canadian Place, Underground city",100
1,"Commerce Court, Victoria Hotel",100
2,"Richmond, Adelaide, King",100
3,"Garden District, Ryerson",100
4,"Harbourfront East, Union Station, Toronto Islands",100


In [27]:
Total = Total.rename(columns={"index":"Neighborhood"})
Total.head()

,Neighborhood,Total
0,"First Canadian Place, Underground city",100
1,"Commerce Court, Victoria Hotel",100
2,"Richmond, Adelaide, King",100
3,"Garden District, Ryerson",100
4,"Harbourfront East, Union Station, Toronto Islands",100


In [28]:
#Drop unnecessary data from the venuesDF dataframe
venuesDF = venuesDF.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude', 'Venue Longitude'], axis=1)

In [29]:
#Take a look to make sure we're all good!
venuesDF.head(10)

,Neighborhood,Venue Category
0,Parkwoods,Park
1,Parkwoods,Pool
2,Parkwoods,Food & Drink Shop
3,Victoria Village,Hockey Arena
4,Victoria Village,Coffee Shop
5,Victoria Village,Portuguese Restaurant
6,Victoria Village,Intersection
7,Victoria Village,Pizza Place
8,"Regent Park, Harbourfront",Bakery
9,"Regent Park, Harbourfront",Coffee Shop


In [30]:
#Find the different unique venue types, and their total number of appearances in the data
venuesDF["Venue Category"].value_counts()

Coffee Shop                        190
Café                                99
Restaurant                          62
Park                                49
Pizza Place                         48
Sandwich Place                      44
Hotel                               43
Italian Restaurant                  41
Japanese Restaurant                 40
Bakery                              37
Bar                                 36
Gym                                 36
Clothing Store                      34
Sushi Restaurant                    30
Grocery Store                       27
Pub                                 27
American Restaurant                 26
Fast Food Restaurant                25
Bank                                24
Gastropub                           22
Breakfast Spot                      22
Pharmacy                            22
Seafood Restaurant                  22
Thai Restaurant                     20
Beer Bar                            19
Ice Cream Shop           

In [59]:
#Find the total number of UNIQUE venue types in each neighborhood, to see how much variety there is
UniqueDF = venuesDF.groupby('Neighborhood')['Venue Category'].nunique().to_frame()

In [60]:
UniqueDF.head(50)

,Venue Category
Neighborhood,
Agincourt,4
"Alderwood, Long Branch",6
"Bathurst Manor, Wilson Heights, Downsview North",19
Bayview Village,4
"Bedford Park, Lawrence Manor East",18
Berczy Park,45
"Birch Cliff, Cliffside West",4
"Brockton, Parkdale Village, Exhibition Place",18
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16


In [63]:
# merge two table on the column "PostalCode"
Total = Total.merge(UniqueDF, on="Neighborhood", how="left")
Total.head(50)

,Neighborhood,Total,Unique,Venue Category_x,Venue Category_y
0,"First Canadian Place, Underground city",100,NaN,53,53
1,"Commerce Court, Victoria Hotel",100,NaN,52,52
2,"Richmond, Adelaide, King",100,NaN,62,62
3,"Garden District, Ryerson",100,NaN,65,65
4,"Harbourfront East, Union Station, Toronto Islands",100,NaN,59,59
5,"Toronto Dominion Centre, Design Exchange",100,NaN,54,54
6,Stn A PO Boxes,96,NaN,59,59
7,St. James Town,86,NaN,53,53
8,Church and Wellesley,78,NaN,54,54
9,"Kensington Market, Chinatown, Grange Park",74,NaN,49,49


In [64]:
#Oops, ran it twice, need to clean it up a bit...
Total = Total.drop(['Unique', 'Venue Category_y'], axis=1)

In [65]:
Total.head()

,Neighborhood,Total,Venue Category_x
0,"First Canadian Place, Underground city",100,53
1,"Commerce Court, Victoria Hotel",100,52
2,"Richmond, Adelaide, King",100,62
3,"Garden District, Ryerson",100,65
4,"Harbourfront East, Union Station, Toronto Islands",100,59


In [69]:
#Clean up that messy column title!
Total.rename(columns={"Venue Category_x":"Unique"}, inplace=True)

In [72]:
Total.head()

,Neighborhood,Total,Unique
0,"First Canadian Place, Underground city",100,53
1,"Commerce Court, Victoria Hotel",100,52
2,"Richmond, Adelaide, King",100,62
3,"Garden District, Ryerson",100,65
4,"Harbourfront East, Union Station, Toronto Islands",100,59


In [73]:
#Create a function to find specific types of venues in specific neighborhoods

def VenueNo(df, neigh, categ):
    return df.query('Neighborhood == @neigh and `Venue Category` == @categ').index.size

In [76]:
VenueNo(venuesDF, 'Garden District, Ryerson', 'Coffee Shop')

8

In [79]:
Neighborhoods = Total["Neighborhood"].to_list()

In [94]:
#Use 'VenueNo' function to create list of total coffee shops in each neighborhood

CoffeeShops = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Coffee Shop')
    CoffeeShops.append(y)

In [95]:
#Use 'VenueNo' function to create list of total grocery stores in each neighborhood
GroceryStores = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Grocery Store')
    GroceryStores.append(y)

In [97]:
#Use 'VenueNo' function to create list of total pharmacies in each neighborhood

Pharmacies = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Pharmacy')
    Pharmacies.append(y)

In [99]:
#Use 'VenueNo' function to create list of total gyms in each neighborhood

Gyms = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Gym')
    Gyms.append(y)

In [100]:
#Use 'VenueNo' function to create list of total banks in each neighborhood

Banks = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Bank')
    Banks.append(y)

In [101]:
#Use 'VenueNo' function to create list of total parks in each neighborhood

Parks = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Park')
    Parks.append(y)

In [102]:
Total["Coffee Shops"] = CoffeeShops

In [103]:
Total.head()

,Neighborhood,Total,Unique,Coffee Shops
0,"First Canadian Place, Underground city",100,53,11
1,"Commerce Court, Victoria Hotel",100,52,13
2,"Richmond, Adelaide, King",100,62,8
3,"Garden District, Ryerson",100,65,8
4,"Harbourfront East, Union Station, Toronto Islands",100,59,13


In [106]:
Total["Grocery Stores"] = GroceryStores
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores
0,"First Canadian Place, Underground city",100,53,11,0
1,"Commerce Court, Victoria Hotel",100,52,13,0
2,"Richmond, Adelaide, King",100,62,8,0
3,"Garden District, Ryerson",100,65,8,0
4,"Harbourfront East, Union Station, Toronto Islands",100,59,13,0


In [109]:
Total["Pharmacies"] = Pharmacies
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies
0,"First Canadian Place, Underground city",100,53,11,0,0
1,"Commerce Court, Victoria Hotel",100,52,13,0,0
2,"Richmond, Adelaide, King",100,62,8,0,0
3,"Garden District, Ryerson",100,65,8,0,0
4,"Harbourfront East, Union Station, Toronto Islands",100,59,13,0,0


In [110]:
Restaurants = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Restaurant')
    Restaurants.append(y)

In [111]:
Total["Banks"] = Banks

In [112]:
Total["Parks"] = Parks
Total["Restaurants"] = Restaurants
Total["Gyms"] = Gyms
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms
0,"First Canadian Place, Underground city",100,53,11,0,0,0,0,4,4
1,"Commerce Court, Victoria Hotel",100,52,13,0,0,0,1,7,4
2,"Richmond, Adelaide, King",100,62,8,0,0,0,0,4,4
3,"Garden District, Ryerson",100,65,8,0,0,1,1,1,1
4,"Harbourfront East, Union Station, Toronto Islands",100,59,13,0,0,0,2,3,1


In [113]:
ChineseRestaurants = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Chinese Restaurant')
    ChineseRestaurants.append(y)

In [115]:
PizzaPlaces = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Pizza Place')
    PizzaPlaces.append(y)

In [129]:
Restaurants = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Restaurant')
    Restaurants.append(y)

In [128]:
Fitness = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Gym / Fitness Center')
    Fitness.append(y)

In [131]:
Gyms = [x + y for x, y in zip(Gyms, Fitness)]

In [133]:
Yoga = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Yoga Studio')
    Yoga.append(y)

In [134]:
Gyms = [x + y for x, y in zip(Gyms, Yoga)]

In [136]:
Total["Gyms"] = Gyms

In [137]:
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms
0,"First Canadian Place, Underground city",100,53,11,0,0,0,0,4,5
1,"Commerce Court, Victoria Hotel",100,52,13,0,0,0,1,7,5
2,"Richmond, Adelaide, King",100,62,8,0,0,0,0,4,5
3,"Garden District, Ryerson",100,65,8,0,0,1,1,1,2
4,"Harbourfront East, Union Station, Toronto Islands",100,59,13,0,0,0,2,3,1


In [144]:
Cafes = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Café')
    Cafes.append(y)

In [145]:
Coffee = [x + y for x, y in zip(CoffeeShops, Cafes)]

In [146]:
Total["CoffeeShops"] = Coffee

In [148]:
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms,CoffeeShops
0,"First Canadian Place, Underground city",100,53,11,0,0,0,0,4,5,18
1,"Commerce Court, Victoria Hotel",100,52,13,0,0,0,1,7,5,20
2,"Richmond, Adelaide, King",100,62,8,0,0,0,0,4,5,13
3,"Garden District, Ryerson",100,65,8,0,0,1,1,1,2,12
4,"Harbourfront East, Union Station, Toronto Islands",100,59,13,0,0,0,2,3,1,17


In [150]:
Total=Total.drop(columns=["CoffeeShops"])

In [151]:
Total["Coffee Shops"] = Coffee

In [152]:
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms
0,"First Canadian Place, Underground city",100,53,18,0,0,0,0,4,5
1,"Commerce Court, Victoria Hotel",100,52,20,0,0,0,1,7,5
2,"Richmond, Adelaide, King",100,62,13,0,0,0,0,4,5
3,"Garden District, Ryerson",100,65,12,0,0,1,1,1,2
4,"Harbourfront East, Union Station, Toronto Islands",100,59,17,0,0,0,2,3,1


In [153]:
Super = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Supermarket')
    Super.append(y)

In [154]:
GroceryStores = [x + y for x, y in zip(GroceryStores, Super)]

In [155]:
Total["Grocery Stores"]=GroceryStores

In [159]:
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms
0,"First Canadian Place, Underground city",100,53,18,0,0,0,0,4,5
1,"Commerce Court, Victoria Hotel",100,52,20,0,0,0,1,7,5
2,"Richmond, Adelaide, King",100,62,13,0,0,0,0,4,5
3,"Garden District, Ryerson",100,65,12,0,0,1,1,1,2
4,"Harbourfront East, Union Station, Toronto Islands",100,59,17,1,0,0,2,3,1


In [160]:
American = []
Sushi = []
Japanese = []
Pizza = []
Sandwich = []
Thai = []
Vietnamese = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'American Restaurant')
    American.append(y)
    s = VenueNo(venuesDF, x, 'Sushi Restaurant')
    Sushi.append(s)
    j = VenueNo(venuesDF, x, 'Japanese Restaurant')
    Japanese.append(j)
    t = VenueNo(venuesDF, x, 'Thai Place')
    Thai.append(t)
    p = VenueNo(venuesDF, x, 'Pizza Place')
    Pizza.append(p)
    w = VenueNo(venuesDF, x, 'Sandwich Place')
    Sandwich.append(w)
    v = VenueNo(venuesDF, x, 'Vietnamese Restaurant')
    Vietnamese.append(v)

In [161]:
Italian = []

for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Italian Restaurant')
    Italian.append(y)

In [162]:
Restaurants = [x + y for x, y in zip(Restaurants, Italian)]

In [163]:
Restaurants = [x + y for x, y in zip(Restaurants, American)]

In [164]:
Total["Restaurants"] = Restaurants

In [165]:
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms
0,"First Canadian Place, Underground city",100,53,18,0,0,0,0,8,5
1,"Commerce Court, Victoria Hotel",100,52,20,0,0,0,1,13,5
2,"Richmond, Adelaide, King",100,62,13,0,0,0,0,6,5
3,"Garden District, Ryerson",100,65,12,0,0,1,1,2,2
4,"Harbourfront East, Union Station, Toronto Islands",100,59,17,1,0,0,2,5,1


In [166]:
Lunch = [x + y for x, y in zip(Pizza, Sandwich)]

In [167]:
Total["Lunch Spots"] = Lunch

In [177]:
Total.head(20)

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms,Lunch Spots,Asian Restaurants
0,"First Canadian Place, Underground city",100,53,18,0,0,0,0,8,5,3,6
1,"Commerce Court, Victoria Hotel",100,52,20,0,0,0,1,13,5,2,3
2,"Richmond, Adelaide, King",100,62,13,0,0,0,0,6,5,2,3
3,"Garden District, Ryerson",100,65,12,0,0,1,1,2,2,3,6
4,"Harbourfront East, Union Station, Toronto Islands",100,59,17,1,0,0,2,5,1,4,3
5,"Toronto Dominion Centre, Design Exchange",100,54,20,0,0,0,0,10,2,2,5
6,Stn A PO Boxes,96,59,14,1,1,0,2,7,3,3,4
7,St. James Town,86,53,12,1,1,0,1,8,2,0,2
8,Church and Wellesley,78,54,9,0,0,0,1,4,3,1,10
9,"Kensington Market, Chinatown, Grange Park",74,49,8,3,1,0,2,0,0,2,4


In [169]:
Japanese = [x + y for x, y in zip(Japanese, Sushi)]

In [170]:
Asian = [x + y for x, y in zip(Japanese, ChineseRestaurants)]

In [172]:
Asian = [x + y for x, y in zip(Asian, Thai)]

In [174]:
Asian = [x + y for x, y in zip(Asian, Vietnamese)]

In [175]:
Total["Asian Restaurants"] = Asian

In [178]:
Bar = []
Beer = []
Pub = []
Gpub = []


for x in Neighborhoods:
    y = VenueNo(venuesDF, x, 'Bar')
    Bar.append(y)
    b = VenueNo(venuesDF, x, 'Beer Bar')
    Beer.append(b)
    p = VenueNo(venuesDF, x, 'Pub')
    Pub.append(p)
    g = VenueNo(venuesDF, x, 'Gastropub')
    Gpub.append(y)

In [179]:
Bar = [x + y for x, y in zip(Bar, Beer)]

In [180]:
Bar = [x + y for x, y in zip(Bar, Pub)]

In [181]:
Restaurants = [x + y for x, y in zip(Restaurants, Gpub)]

In [182]:
Total["Bar"] = Bar
Total["Restaurants"] = Restaurants

In [183]:
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms,Lunch Spots,Asian Restaurants,Bar
0,"First Canadian Place, Underground city",100,53,18,0,0,0,0,10,5,3,6,5
1,"Commerce Court, Victoria Hotel",100,52,20,0,0,0,1,15,5,2,3,5
2,"Richmond, Adelaide, King",100,62,13,0,0,0,0,9,5,2,3,3
3,"Garden District, Ryerson",100,65,12,0,0,1,1,3,2,3,6,2
4,"Harbourfront East, Union Station, Toronto Islands",100,59,17,1,0,0,2,7,1,4,3,3


In [243]:
Total = Total.rename(columns={"Bar":"Bars"})

In [197]:
from sklearn import preprocessing

In [199]:
a = Restaurants
amin, amax = min(a), max(a)
for i, val in enumerate(a):
    a[i] = (val-amin) / (amax-amin)

In [201]:
Total["Restaurants"] = a

In [211]:
b = TotalL
amin, amax = min(b), max(b)
for i, val in enumerate(b):
    b[i] = (val-amin) / (amax-amin)

In [207]:
TotalL = Total["Total"].to_numpy()

In [209]:
TotalL = TotalL.tolist()

In [213]:
Total["Total"] = TotalL

In [219]:
Unique = Total["Unique"].to_numpy()

In [220]:
Unique = Unique.tolist()

In [221]:
c = Unique
amin, amax = min(c), max(c)
for i, val in enumerate(c):
    c[i] = (val-amin) / (amax-amin)

In [222]:
Total["Unique"] = Unique

In [224]:
d = CoffeeShops
amin, amax = min(d), max(d)
for i, val in enumerate(d):
    d[i] = (val-amin) / (amax-amin)

In [225]:
Total["Coffee Shops"] = d

In [227]:
e = GroceryStores
amin, amax = min(e), max(e)
for i, val in enumerate(e):
    e[i] = (val-amin) / (amax-amin)

In [228]:
Total["Grocery Stores"] = e

In [230]:
f = Pharmacies
amin, amax = min(f), max(f)
for i, val in enumerate(f):
    f[i] = (val-amin) / (amax-amin)

In [231]:
Total["Pharmacies"] = f

In [233]:
g = Banks
amin, amax = min(g), max(g)
for i, val in enumerate(g):
    g[i] = (val-amin) / (amax-amin)

In [234]:
Total["Banks"] = g

In [236]:
h = Parks
amin, amax = min(h), max(h)
for i, val in enumerate(h):
    h[i] = (val-amin) / (amax-amin)
    
Total["Parks"] = h

In [238]:
j = Gyms
amin, amax = min(j), max(j)
for i, val in enumerate(j):
    j[i] = (val-amin) / (amax-amin)
    
Total["Gyms"] = j

In [240]:
k = Lunch
amin, amax = min(k), max(k)
for i, val in enumerate(k):
    k[i] = (val-amin) / (amax-amin)
    
Total["Lunch Spots"] = k

In [241]:
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms,Lunch Spots,Asian Restaurants,Bar
0,"First Canadian Place, Underground city",1.0,0.812500,0.785714,0.00,0.0,0.0,0.000000,0.666667,1.0,0.6,6,5
1,"Commerce Court, Victoria Hotel",1.0,0.796875,0.928571,0.00,0.0,0.0,0.333333,1.000000,1.0,0.4,3,5
2,"Richmond, Adelaide, King",1.0,0.953125,0.571429,0.00,0.0,0.0,0.000000,0.600000,1.0,0.4,3,3
3,"Garden District, Ryerson",1.0,1.000000,0.571429,0.00,0.0,0.5,0.333333,0.200000,0.4,0.6,6,2
4,"Harbourfront East, Union Station, Toronto Islands",1.0,0.906250,0.928571,0.25,0.0,0.0,0.666667,0.466667,0.2,0.8,3,3


In [242]:
l = Asian
amin, amax = min(l), max(l)
for i, val in enumerate(l):
    l[i] = (val-amin) / (amax-amin)
    
Total["Asian Restaurants"] = l
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms,Lunch Spots,Asian Restaurants,Bar
0,"First Canadian Place, Underground city",1.0,0.812500,0.785714,0.00,0.0,0.0,0.000000,0.666667,1.0,0.6,0.6,5
1,"Commerce Court, Victoria Hotel",1.0,0.796875,0.928571,0.00,0.0,0.0,0.333333,1.000000,1.0,0.4,0.3,5
2,"Richmond, Adelaide, King",1.0,0.953125,0.571429,0.00,0.0,0.0,0.000000,0.600000,1.0,0.4,0.3,3
3,"Garden District, Ryerson",1.0,1.000000,0.571429,0.00,0.0,0.5,0.333333,0.200000,0.4,0.6,0.6,2
4,"Harbourfront East, Union Station, Toronto Islands",1.0,0.906250,0.928571,0.25,0.0,0.0,0.666667,0.466667,0.2,0.8,0.3,3


In [246]:
m = Bar
amin, amax = min(m), max(m)
for i, val in enumerate(l):
    m[i] = (val-amin) / (amax-amin)
    
Total["Bars"] = m
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms,Lunch Spots,Asian Restaurants,Bars
0,"First Canadian Place, Underground city",1.0,0.812500,0.785714,0.00,0.0,0.0,0.000000,0.666667,1.0,0.6,0.6,0.10
1,"Commerce Court, Victoria Hotel",1.0,0.796875,0.928571,0.00,0.0,0.0,0.333333,1.000000,1.0,0.4,0.3,0.05
2,"Richmond, Adelaide, King",1.0,0.953125,0.571429,0.00,0.0,0.0,0.000000,0.600000,1.0,0.4,0.3,0.05
3,"Garden District, Ryerson",1.0,1.000000,0.571429,0.00,0.0,0.5,0.333333,0.200000,0.4,0.6,0.6,0.10
4,"Harbourfront East, Union Station, Toronto Islands",1.0,0.906250,0.928571,0.25,0.0,0.0,0.666667,0.466667,0.2,0.8,0.3,0.05


In [253]:
#Grocery stores are one of the most important things you can have in your neighborhood for convenience, so we're going to give them added importance
Total["Grocery Stores"] = Total["Grocery Stores"]*5

In [255]:
#Banks and Pharmacies are also less common and more important than restaurants, so we're going to give them added value as well

Total["Banks"] = Total["Banks"]*2
Total["Pharmacies"] = Total["Pharmacies"]*2

In [257]:
#Because Parks are more of a luxury, and a few neighborhoods with parks are getting a big advantage, we're going to lessen the value of parks

Total["Parks"] = Total["Parks"]/2

In [263]:
#Sum all the values of each row to find the total Neighborhood Convenience Score, or NCS
Total["NCS"] = Total.sum(axis=1)
Total.head()

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms,Lunch Spots,Asian Restaurants,Bars,NCS
0,"First Canadian Place, Underground city",1.0,0.812500,0.785714,0.00,0.0,0.0,0.000000,0.666667,1.0,0.6,0.6,0.10,11.129762
1,"Commerce Court, Victoria Hotel",1.0,0.796875,0.928571,0.00,0.0,0.0,0.166667,1.000000,1.0,0.4,0.3,0.05,11.284226
2,"Richmond, Adelaide, King",1.0,0.953125,0.571429,0.00,0.0,0.0,0.000000,0.600000,1.0,0.4,0.3,0.05,9.749107
3,"Garden District, Ryerson",1.0,1.000000,0.571429,0.00,0.0,1.0,0.166667,0.200000,0.4,0.6,0.6,0.10,11.276190
4,"Harbourfront East, Union Station, Toronto Islands",1.0,0.906250,0.928571,1.25,0.0,0.0,0.333333,0.466667,0.2,0.8,0.3,0.05,12.469643


In [262]:
#Find the most convenient neighborhoods in the Toronto area!

Total.sort_values(by=['NCS'], ascending=False)

,Neighborhood,Total,Unique,Coffee Shops,Grocery Stores,Pharmacies,Banks,Parks,Restaurants,Gyms,Lunch Spots,Asian Restaurants,Bars,NCS
9,"Kensington Market, Chinatown, Grange Park",0.737374,0.750000,0.285714,3.75,1.0,0.0,0.333333,0.266667,0.0,0.4,0.4,0.066667,7.989755
6,Stn A PO Boxes,0.959596,0.906250,0.714286,1.25,1.0,0.0,0.333333,0.466667,0.6,0.6,0.4,0.066667,7.296798
27,Thorncliffe Park,0.212121,0.296875,0.071429,2.50,1.0,1.0,0.166667,0.066667,0.4,0.6,0.0,0.000000,6.313758
42,"Dufferin, Dovercourt Village",0.131313,0.171875,0.000000,2.50,2.0,1.0,0.166667,0.066667,0.2,0.0,0.0,0.000000,6.236521
4,"Harbourfront East, Union Station, Toronto Islands",1.000000,0.906250,0.928571,1.25,0.0,0.0,0.333333,0.466667,0.2,0.8,0.3,0.050000,6.234821
20,Leaside,0.323232,0.375000,0.285714,2.50,0.0,2.0,0.000000,0.066667,0.0,0.2,0.1,0.016667,5.867280
33,Christie,0.161616,0.156250,0.071429,5.00,0.0,0.0,0.333333,0.133333,0.0,0.0,0.0,0.000000,5.855961
7,St. James Town,0.858586,0.812500,0.428571,1.25,1.0,0.0,0.166667,0.533333,0.4,0.0,0.2,0.033333,5.682991
39,Downsview,0.141414,0.171875,0.071429,3.75,0.0,1.0,0.333333,0.000000,0.2,0.0,0.0,0.000000,5.668051
15,"St. James Town, Cabbagetown",0.414141,0.515625,0.214286,1.25,1.0,1.0,0.166667,0.266667,0.0,0.6,0.2,0.033333,5.660719
